In [ ]:
#Install required packages

!pip install nltk
!pip install transformers

In [ ]:

# Import all imporatant packages

from nltk import sent_tokenize
import nltk
nltk.download('punkt')
import pandas as pd
import ast
import glob
from os.path import join

from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

In [ ]:
#Load the Falcon-40b-instruct model

model = "tiiuae/falcon-40b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

In [4]:
#Load all the annotation files from annotation_data folder from github: "/data/sampled_data"

annotation_files= glob.glob(join("/home/mihirp/sampled_data", "*"))

# Provide where you want to save generated summaries

save_data= '/home/mihirp/annotation_processed'

In [ ]:
#Loop for generating model summaries

for file in annotation_files:
    filename= file.split('/')[-1]
    print(filename)
    data_df= pd.read_csv(file)
    print(data_df.head(5))

    final_results=[]

    for index, row in data_df.iterrows():
        print("Running for the index " + str(index+1))

        prompt= "You are an extractive summarizer. You are presented with a document. The document is a collection of sentences and each sentence is numbered with sentence ids. Understand the given document and create a meaningful summary by picking sentences from document. Please list the sentence ids as output so that sentences corresponding to the generated ids summarize the document coherently. Learn from the below example:\r\n\r\nDocument:\r\n1. Olympic gold medallist Jessica Ennis-Hill has confirmed she will return to competition in London this July following her break from athletics to become a mother.\r\n2. Ennis-Hill provided one of London 2012's most captivating storylines by surging to heptathlon gold, and the Sheffield-born star will return to the Olympic Stadium three years on to compete in the Sainsbury's Anniversary Games.\r\n3. The 29-year-old has not competed since the same event in 2013 and gave birth to her son, Reggie, last summer.\r\n4. But her return to action sets up the prospect of a showdown against Katarina Johnson-Thompson, the brilliant young British heptathlete who is the heir to Ennis-Hill's throne.\r\n5. Jessica Ennis-Hill became a national hero when she won heptathlon gold at the London 2012 Olympics .\r\n6. Ennis-Hill has not competed since 2013 as she took time off to become a mother .\r\n7. 'I am really looking forward to it,' said Ennis-Hill, who could compete in the long jump or 110m hurdles.\r\n8. 'My race schedule is starting to starting to take shape and it will be good to compete and get a sense of where I am in my return to competing.\r\n9. 'My main goal this season is to be as competitive as possible with the long-term goal being the Rio Olympics.\r\n10. 'Diamond League meetings always have the best athletes in the world so I\u2019m sure this will be a good test for me; I want to perform well being back on the big stage in London but I will be realistic as 2015 is about the challenge of getting back to competitive shape after having my little boy and ultimately making the necessary progression to be at my best for Rio.'\r\n11. Katarina Johnson-Thompson has emerged as the new rising star of British athletics and Ennis-Hill's heir .\r\n12. Another British hero of London 2012, Mo Farah, will also compete in the Diamond League event, which serves as a warm-up for the World Championships in Beijing in August.\r\n13. Ennis-Hill will take part in the two-day meeting on July 24 and 25, with the Sainsbury's IPC Athletics Grand Prix Final taking place on July 26.\r\n14. Ennis-Hill added: 'The 2012 Olympics were an incredible experience for me and it will be very special to step out on that track again.\r\n15. It will be amazing to compete in front of all our British fans who I am sure will have their own memories of the London Games too.\r\n\r\nSummary: <s> [2, 5, 6, 11, 12, 15]\r\n\r\nDocument:\r\n"
        sentences = sent_tokenize(row.input)

        collective_sent= ""
        for i, sent in enumerate(sentences):
            temp= str(i+1) + '. ' + sent + '\n'
            collective_sent+= temp

        ip = prompt + collective_sent + '\nPlease Create a concise summary using as few sentences as possible.\nSummary: <s>'

        count=0
        while True:
            preprocessed = pipeline.preprocess(ip)
            model_outputs = pipeline.forward(preprocessed, max_length=2048)
            outputs = pipeline.postprocess(model_outputs)

            try:
                summary= outputs[0]['generated_text'].split(' <s>')[-1]
                sent_list = ast.literal_eval(summary.strip())
                print(type(sent_list), sent_list)
                break

            except:
                count+=1
                if count>10:
                    print("Not able to generate good model summary, try again! Hence, giving random output.")
                    sent_list = [1,2,3,4]
                    break
                continue

        final_summary= str()
        
        try:
            for sent_id in sent_list:
                s= sentences[sent_id-1] + ' '
                final_summary += s
        except:
            for sent_id in [1]:
                s= sentences[sent_id-1] + ' '
                final_summary += s

        print(final_summary)

        new_data= {'type': row.type, 'text': row.input, 'summary': final_summary}
        final_results.append(new_data)

        result_df= pd.DataFrame(final_results)

        #Provide a path to save processed data

        result_df.to_csv(join(save_data, filename), index=False)